# 重新解压数据集

In [8]:
! ls -a /mnt/liuwenrui/dataset/multi30k-dataset/data/speech/16khz_wav/fr/train | grep 25092

ls: cannot access '/mnt/liuwenrui/dataset/multi30k-dataset/data/speech/16khz_wav/fr/train': No such file or directory


In [6]:
import os
import pandas as pd
import tqdm
from pathlib import Path

# tsv_root = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/format_data/fr-en_enhanced"
# tsv_root = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/format_data/fr-en_enhanced/fr_source_unit"
tsv_root = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/format_data/fr-en"
# tsv_root = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/format_data/fr-en/fr_source_unit"
for tsv in tqdm.tqdm(os.listdir(tsv_root)):
    if not tsv.endswith(".tsv"):
        continue
    tsv_path = os.path.join(tsv_root, tsv)
    src_root = "/mnt/liuwenrui/dataset"
    tgt_root = "/opt/data/private/dsy/project/dataset"
    tsv = pd.read_csv(tsv_path, sep='\t')
    # audio_list = tsv["audio"].tolist()
    audio_list = tsv["src_audio"].tolist()
    for i in range(len(audio_list)):
        audio_list[i] = audio_list[i].replace(src_root, tgt_root)
    # tsv["audio"] = audio_list
    tsv["src_audio"] = audio_list
    tsv.set_index("id", inplace=True)
    # print(666, tsv.columns)
    # tsv.drop(columns="Uname: 0", inplace=True)
    # print(tsv)
    tsv.to_csv(tsv_path, sep='\t')

100%|██████████| 10/10 [00:00<00:00, 15.84it/s]


# 清理checkpoints

In [1]:
import os
import shutil
# ckpt_dir = "/root/autodl-tmp/liuwenrui/project/model/multimodal_S2UT/mm_s2ut/checkpoints/enhanced_fr-en"
ckpt_dir = "/root/autodl-tmp/liuwenrui/project/model/multimodal_S2UT/mm_s2ut/checkpoints/enhanced_fr-en_mm"
for file in os.listdir(ckpt_dir):
    if file.endswith(".pt") and ("best" not in file and "last" not in file):
        os.remove(os.path.join(ckpt_dir, file))

In [1]:
import os
import shutil
ckpt_dir = "/root/autodl-tmp/liuwenrui/project/model/multimodal_S2UT/mm_s2ut/checkpoints/enhanced_fr-en"
# ckpt_dir = "/root/autodl-tmp/liuwenrui/project/model/multimodal_S2UT/mm_s2ut/checkpoints/enhanced_fr-en_mm_dropout"
for file in os.listdir(ckpt_dir):
    if file.endswith(".pt") and ("best" not in file and "last" not in file):
        num = file.split('.')[0][len("checkpoint"):]
        num = int(num)
        if num <= 53:
            os.remove(os.path.join(ckpt_dir, file))

# 统计音频时长

In [2]:
import os, pydub, tqdm

audio_root = "/root/autodl-tmp/liuwenrui/project/dataset/multi30k-dataset/data/speech/16khz_wav"
tot_duration = 0.0
tot_item = 0
for dir in tqdm.tqdm(os.listdir(audio_root)):
    audio_dir = os.path.join(audio_root, dir)
    for file in os.listdir(audio_dir):
        if not file.endswith(".wav"):
            continue
        audio_path = os.path.join(audio_dir, file)
        audio = pydub.AudioSegment.from_wav(audio_path)
        tot_duration += audio.duration_seconds
        tot_item += 1
print(f"total duration = {tot_duration}")
print(f"total item = {tot_item}")

100%|██████████| 12/12 [00:24<00:00,  2.06s/it]

total duration = 277460.8559999971
total item = 64950


In [3]:
print(f"total duration = {tot_duration}")
print(f"total item = {tot_item}")

total duration = 277460.8559999971
total item = 64950


In [4]:
277460.8559999971 / 60

4624.347599999952

In [5]:
277460.8559999971 / 60 / 60

77.0724599999992

In [3]:
import torch
pth_path = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/image_feat/vit_base_patch16_384/test2.pth"
pth = torch.load(pth_path)
print(len(pth))

461


# load vit from mm_s2ut checkpoint

### load multimodal_S2UT task

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch, argparse
import fairseq
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

user_dir = "/opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut"
fairseq.utils.import_user_module(argparse.Namespace(user_dir=user_dir))

model_dir = "/opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut/checkpoints/textless_en-fr"
model_path = f"{model_dir}/checkpoints/checkpoint_best.pt"
state = fairseq.checkpoint_utils.load_checkpoint_to_cpu(model_path)
task = fairseq.tasks.setup_task(state["cfg"]["task"], from_checkpoint=True)
model = task.build_model(state["cfg"]["model"], from_checkpoint=True)
model.load_state_dict(state["model"], strict=True, model_cfg=state["cfg"]["model"])
model.to(device)
generator = task.build_generator([model], state["cfg"]["task"])

/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-28 04:49:29 | INFO | fairseq.tasks.speech_to_speech | dictionary size: 1,004
2023-07-28 04:49:30 | INFO | mm_s2ut.models.mm_s2s_transformer | load_visual_extractor_type = None
2023-07-28 04:49:30 | INFO | mm_s2ut.models.mm_s2s_transformer | multimodal_translation_flag = False
2023-07-28 04:49:30 | INFO | mm_s2ut.models.mm_s2s_transformer | is_fusion_top = False


### load multimodal_S2UT valid dataset

In [2]:
# model.endocer
# dir(model)
# encoder = getattr(model, "encoder")
# vit = encoder.vit
subset = "valid"
# subset = "train"
task.load_dataset(subset)
dataset = task.dataset(subset)

2023-07-28 04:49:39 | WARNING | fairseq.data.audio.data_cfg | Auto converting transforms into feature_transforms, but transforms will be deprecated in the future. Please update this in the config.
2023-07-28 04:49:39 | INFO | fairseq.data.audio.speech_to_text_dataset | 'valid' has 0.00% OOV
2023-07-28 04:49:39 | INFO | fairseq.data.audio.speech_to_text_dataset | MultiModalSpeechToSpeechMultitaskDataset(split="valid", n_samples=1_014, prepend_tgt_lang_tag=False, n_frames_per_step=1, shuffle=False, feature_transforms=CompositeAudioFeatureTransform(
    UtteranceCMVN(norm_means=True, norm_vars=True)
), waveform_transforms=None, dataset_transforms=CompositeAudioDatasetTransform(
))
2023-07-28 04:49:39 | INFO | mm_s2ut.data.speech_to_speech_dataset | num of noise_wav = 0
2023-07-28 04:49:39 | INFO | mm_s2ut.data.speech_to_speech_dataset | noise_prob = 0.0
2023-07-28 04:49:39 | INFO | mm_s2ut.data.speech_to_speech_dataset | noise_snr = 0.0
2023-07-28 04:49:39 | INFO | mm_s2ut.data.speech_to_

In [13]:
import torch, torchaudio
import json
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import IPython.display as ipd
from fairseq import hub_utils
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.speech_to_text.hub_interface import S2THubInterface
from fairseq.models.text_to_speech import CodeHiFiGANVocoder
from fairseq.models.text_to_speech.hub_interface import VocoderHubInterface

unit_hifigan_dir = "/opt/data/private/dsy/project/checkpoint/unit_hifigan_mhubert_vp_en_es_fr_it3_400k_layer11_km1000_lj_dur"
unit_hifigan_config = hub_utils.from_pretrained(
    unit_hifigan_dir,
    "model.pt",
    ".",
    archive_map=CodeHiFiGANVocoder.hub_models(),
    config_yaml="config.json",
    fp16=False,
    is_vocoder=True,
)

# unit_hifigan_dir = "/opt/data/private/dsy/project/checkpoint/unit_hifigan/fr"
# unit_hifigan_config = hub_utils.from_pretrained(
#     unit_hifigan_dir,
#     "g_00500000",
#     ".",
#     archive_map=CodeHiFiGANVocoder.hub_models(),
#     config_yaml="config.json",
#     fp16=False,
#     is_vocoder=True,
# )

with open(f"{unit_hifigan_config['args']['data']}/config.json") as f:
    vocoder_cfg = json.load(f)
assert (
    len(unit_hifigan_config["args"]["model_path"]) == 1
), "Too many vocoder models in the input"
vocoder = CodeHiFiGANVocoder(unit_hifigan_config["args"]["model_path"][0], vocoder_cfg)
unit_hifigan = VocoderHubInterface(vocoder_cfg, vocoder)

# wav2vec2_dir = "/opt/data/private/dsy/project/checkpoint/wav2vec2-large-960h-lv60-self"
# wav2vec2_dir = "/opt/data/private/dsy/project/checkpoint/wav2vec2-base-10k-voxpopuli-ft-en"
wav2vec2_dir = "/opt/data/private/dsy/project/checkpoint/wav2vec2-base-960h"
# wav2vec2_dir = "/opt/data/private/dsy/project/checkpoint/jonatasgrosman/wav2vec2-large-fr-voxpopuli-french"
# wav2vec2_dir = "/opt/data/private/dsy/project/checkpoint/wav2vec2-base-10k-voxpopuli-ft-fr"
# wav2vec2_dir = "/opt/data/private/dsy/project/checkpoint/jonatasgrosman/wav2vec2-large-xlsr-53-spanish"
wav2vec2_processor = Wav2Vec2Processor.from_pretrained(wav2vec2_dir)
wav2vec2 = Wav2Vec2ForCTC.from_pretrained(wav2vec2_dir)
wav2vec2.to(device)
print()

2023-07-28 04:58:46 | INFO | fairseq.file_utils | loading archive file /opt/data/private/dsy/project/checkpoint/unit_hifigan_mhubert_vp_en_es_fr_it3_400k_layer11_km1000_lj_dur
2023-07-28 04:58:47 | INFO | fairseq.models.text_to_speech.vocoder | loaded CodeHiFiGAN checkpoint from /opt/data/private/dsy/project/checkpoint/unit_hifigan_mhubert_vp_en_es_fr_it3_400k_layer11_km1000_lj_dur/model.pt


Removing weight norm...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /opt/data/private/dsy/project/checkpoint/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### data item

In [14]:
index = 6
item = dataset[index][0]

### source en wav

In [15]:
# wav_path = item.src_audio_path.replace("/fr", "/en")
wav_path = item.src_audio_path
# wav_path = item.src_audio_path.replace("/en", "/fr")
print(wav_path)
audio, sr = torchaudio.load(wav_path)
wav = audio[0]
input_values = wav2vec2_processor(wav, sampling_rate=sr, return_tensors="pt", padding="longest").input_values
input_values = input_values.to(device)
logits = wav2vec2(input_values).logits
logits = logits.cpu()
predicted_ids = torch.argmax(logits, dim=-1)
transcript = wav2vec2_processor.batch_decode(predicted_ids)
transcript = transcript[0]
transcript = transcript.lower()
print(transcript)

/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/en/valid/1.wav
a group of men are loading cotton on to a track


### synthesis target code_unit

In [16]:
# code = item.target.detach().clone().cpu().numpy()
code = dataset[index][1]["source_unit"].detach().clone().cpu().numpy()
unit = ' '.join([str(i - 4) for i in code])
# code = [63, 644, 254, 512, 2, 416, 426, 677, 253, 268, 876, 417, 541, 114, 65, 43, 460, 652, 394, 327, 635, 803, 896, 627, 168, 143, 912, 431, 531, 668, 462, 125, 255, 722, 11, 884, 397, 441, 624, 319, 263, 45, 272, 119, 607, 687, 660, 351, 557, 75, 788, 628, 174, 425, 820, 397, 364, 696, 845, 598, 574, 200, 512, 409, 263, 914, 445, 137, 848, 275, 933, 748, 12, 977, 950, 74, 492, 611, 916, 309, 584, 902, 819, 415, 497, 418]
# unit = ' '.join([str(i) for i in code])
# unit = "63 463 991 187 594 725 409 416 426 677 253 268 876 304 585 37 623 717 43 988 327 905 635 908 816 325 765 491 500 873 23 58 700 955 798 170 863 474 778 263 914 445 137 474 980 134 688 350 836 238 252 125 816 886 239 304 585 483 905 8 667 214 416 968 402 687 660 166 549 908 211 620 112 428 46 812 222 62 725 409 914 445 137 474 954 748 12 977 950 113 920 635 377 611 385 309 584 603 584 819 415 497 418"
print(unit)
tts_sample = unit_hifigan.get_model_input(unit)
wav, sr = unit_hifigan.get_prediction(tts_sample)
ipd.Audio(wav, rate=sr)

63 264 644 254 565 734 263 45 416 119 426 647 663 713 268 876 932 148 889 338 877 384 907 488 620 385 659 714 609 710 254 27 579 969 925 781 645 908 380 660 555 398 212 455 143 38 914 445 119 469 167 655 246 764 650 620 352 301 654 659 878 714 609 710 254 493 361 274 79 799 220 483 565 734 793 105 777 326 728 227 647 167 761 907 430 70 219 506 107 417 755 902 415 772 497 418 -2


### transcript

In [17]:
input_values = wav2vec2_processor(wav, sampling_rate=sr, return_tensors="pt", padding="longest").input_values
input_values = input_values.to(device)
logits = wav2vec2(input_values).logits
logits = logits.cpu()
predicted_ids = torch.argmax(logits, dim=-1)
transcript = wav2vec2_processor.batch_decode(predicted_ids)
transcript = transcript[0]
transcript = transcript.lower()
print(transcript)

a grup of man are loading kun on to a track


In [12]:
input_values = wav2vec2_processor(wav, sampling_rate=sr, return_attention_mask=True, return_tensors="pt", padding="longest")
input_values["attention_mask"]

tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)

### image

In [ ]:
from IPython import display
display.display(display.Image(filename=item.img_path))

In [19]:
import timm
# vit = encoder.vit
checkpoint_path = "/opt/data/private/dsy/project/checkpoint/vit_base_patch16_clip_384.laion2b_ft_in12k_in1k/pytorch_model.bin"
vit = timm.create_model(
    "vit_base_patch16_clip_384.laion2b_ft_in12k_in1k",
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)
# timm.models.load_checkpoint(vit, checkpoint_path, strict=False)
vit = vit.to('cuda:0')
vit = vit.eval()

2023-06-07 13:07:15 | INFO | timm.models._builder | Loading pretrained weights from Hugging Face hub (timm/vit_base_patch16_clip_384.laion2b_ft_in12k_in1k)
2023-06-07 13:07:15 | INFO | timm.models._hub | [timm/vit_base_patch16_clip_384.laion2b_ft_in12k_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


In [28]:
vit = encoder.vit

In [96]:
import timm, PIL
from extract_feature import vit_rollout
data_config = timm.data.resolve_model_data_config(vit)
# data_config["mean"] = (0, 0, 0)
# data_config["std"] = (1, 1, 1)
transforms = timm.data.create_transform(**data_config, is_training=False)
img = PIL.Image.open(item.img_path).convert("RGB")
inputs = transforms(img).unsqueeze(0)
inputs = inputs.to('cuda:0')
grad_rollout = vit_rollout.VITAttentionRollout(vit)
mask = grad_rollout(inputs)

In [58]:
print(img.size)
print(inputs.shape)
print(mask.shape)
# print(mask)

(500, 404)
torch.Size([1, 3, 384, 384])
(24, 24)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def show_mask_on_image(img, mask, approach="original"):
    if approach == "original":
        img = np.float32(img) / 255
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255
        cam = heatmap + np.float32(img)
        cam = cam / np.max(cam)
        return np.uint8(255 * cam)
    if approach == "add":
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        cam = heatmap + np.float32(img)
        return cam
    if approach == "multiply":
        img = np.float32(img) / 255
        cam = cv2.multiply(img, mask)
        return cam
 
# mask = cv2.resize(mask, (384, 384))
# mask = np.zeros_like(mask)
new_mask = show_mask_on_image(inputs[0].permute(1, 2, 0).cpu(), mask, "add")
plt.imshow(new_mask)
# plt.imshow(img)
# plt.imshow(inputs[0].cpu()[2, :, :])
# plt.imshow(inputs[0].permute(1, 2, 0).cpu())
plt.show()

In [ ]:
plt.imshow(mask)
plt.show()
inputs[0].permute(1, 2, 0).cpu()

In [ ]:
plt.imshow(inputs[0].permute(1, 2, 0).cpu())
plt.show()

In [88]:
# index = 0
# item = dataset[index][0]
item.src_audio_path
item.img_path
item.source
item.target
item.src_audio_path
type(img)

PIL.Image.Image

In [3]:
import torch
# text_mask = [b, t1]
# img_mask = [b, t2]
# text_img_mask = [b, t1, t2]
a = torch.rand(7, 259) # (7, 259)
a = a[:, :, None] # (7, 259, 1)
a = a.repeat(1, 1, 577) # (7, 259, 577)
print(a.shape)

def forward_padding_mask(
    features: torch.Tensor,
    padding_mask: torch.Tensor,
) -> torch.Tensor:
    extra = padding_mask.size(1) % features.size(1)
    if extra > 0:
        padding_mask = padding_mask[:, :-extra]
    padding_mask = padding_mask.view(padding_mask.size(0), features.size(1), -1)
    padding_mask = padding_mask.all(-1)
    return padding_mask

padding_mask = forward_padding_mask(torch.rand(7, 259, 768), a)
print(padding_mask.shape)

torch.Size([7, 259, 577])
torch.Size([7, 259])


In [1]:
import torch

def forward_padding_mask(
    features: torch.Tensor,
    padding_mask: torch.Tensor,
) -> torch.Tensor:
    extra = padding_mask.size(1) % features.size(1)
    if extra > 0:
        padding_mask = padding_mask[:, :-extra]
    padding_mask = padding_mask.view(padding_mask.size(0), features.size(1), -1)
    padding_mask = padding_mask.all(-1)
    return padding_mask



/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.12.2


In [ ]:
cd /opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut
cp -ar /opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut `ls | grep -v /opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut/checkpoints | xargs` /opt/data/private/dsy/project/opt/code/

cp -ar `ls | grep -v checkpoints | xargs` /opt/data/private/dsy/project/opt/code/mm_s2ut
cp -ar /opt/data/private/dsy/project/model/multimodal_S2UT/fairseq /opt/data/private/dsy/project/opt/code/

In [3]:
import os
txt_path = "/opt/data/private/dsy/project/dataset/flickr30k/test_2017_mscoco.txt"
lines = []
with open(txt_path, mode="r+") as f:
    for line in f.readlines():
        line = line.strip()
        if len(line):
            line = line.split('#')[0]
            lines.append(line)
with open(txt_path, mode="w+") as f:
    f.write('\n'.join(lines))

# 查看max-source-positions、max-target-positions、target-code-size

# check dataloader

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch, argparse
import fairseq
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_dir = "/opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut/checkpoints/textless_fr-en_mm_dropout_vit_"
user_dir = os.path.join(model_dir, "code/mm_s2ut")
fairseq.utils.import_user_module(argparse.Namespace(user_dir=user_dir))

model_path = os.path.join(model_dir, "checkpoints/checkpoint_best.pt")
state = fairseq.checkpoint_utils.load_checkpoint_to_cpu(model_path)
task = fairseq.tasks.setup_task(state["cfg"]["task"], from_checkpoint=True)
model = task.build_model(state["cfg"]["model"], from_checkpoint=True)
model.load_state_dict(state["model"], strict=True, model_cfg=state["cfg"]["model"])
model.to(device)
generator = task.build_generator([model], state["cfg"]["task"])

/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-15 13:22:30 | INFO | fairseq.tasks.speech_to_speech | dictionary size: 1,004
2023-06-15 13:22:31 | INFO | mm_s2ut.models.mm_s2s_transformer | load_visual_extractor_type = vit
2023-06-15 13:22:31 | INFO | mm_s2ut.models.mm_s2s_transformer | path of visual extractor is /opt/data/private/dsy/project/checkpoint/vit_base_patch16_clip_384.laion2b_ft_in12k_in1k/pytorch_model.bin
2023-06-15 13:22:32 | INFO | timm.models._builder | Loading pretrained weights from Hugging Face hub (timm/vit_base_patch16_clip_384.laion2b_ft_in12k_in1k)
2023-06-15 13:22:32 | INFO | timm.models._hub | [timm/vit_base_patch16_clip_384.laion2b_ft_in12k_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors

In [2]:
# subset = "train"
# subset = "valid"
subset = "test.2017"
task.load_dataset(subset)
dataset = task.dataset(subset)

2023-06-15 13:22:57 | INFO | mm_s2ut.tasks.speech_to_speech | 666 image_feat_path = ['/opt/data/private/dsy/project/dataset/multi30k-dataset/data/image_feat/vit_base_patch16_384']
2023-06-15 13:22:58 | INFO | mm_s2ut.data.speech_to_speech_dataset | <mm_s2ut.data.speech_to_speech_dataset.ImageDataset object at 0x7f73e01ea0d0> load 1000 image features
2023-06-15 13:22:58 | INFO | mm_s2ut.data.speech_to_speech_dataset | 666 img_path_list = 1000
2023-06-15 13:22:58 | INFO | mm_s2ut.data.speech_to_speech_dataset | 666 img_dir = /opt/data/private/dsy/project/dataset/flickr30k/test2017-images
2023-06-15 13:22:58 | WARNING | fairseq.data.audio.data_cfg | Auto converting transforms into feature_transforms, but transforms will be deprecated in the future. Please update this in the config.
2023-06-15 13:22:58 | INFO | fairseq.data.audio.speech_to_text_dataset | 'test.2017' has 0.00% OOV
2023-06-15 13:22:58 | INFO | fairseq.data.audio.speech_to_text_dataset | MultiModalSpeechToSpeechMultitaskDatas

In [3]:
item_list = []
source_unit_list = []
inputs = []
# n_item = [0, 1, 2]
n_item = [5, 6, 7]
for i in n_item:
    item, source_unit = dataset[i]
    item_list.append(item)
    avg = {
        "source": item.source.mean(-1).mean(), 
        "src_audio_path": item.src_audio_path, 
        "target": item.target.float().mean(), 
        "img_tensor": item.img_tensor.mean(-1).mean(), 
        "img_list": item.img_list[0].mean(-1).mean(), 
        "img_path": item.img_path, 
        "source_unit": source_unit["source_unit"].float().mean(), 
    }
    inputs.append(avg)
    source_unit_list.append(source_unit)
for i in range(len(inputs)):
    print(n_item[i], inputs[i])
print()

collate_dic = dataset.collater([[item_list[i], source_unit_list[i]] for i in range(len(item_list))])
collate_item_list = []
collate_inputs = []
src_lengths = collate_dic["net_input"]["src_lengths"].cpu().numpy().tolist()
target_lengths = collate_dic["target_lengths"].cpu().numpy().tolist()
ids = collate_dic["id"].cpu().numpy().tolist()
order = collate_dic["order"].cpu().numpy().tolist()
for i in range(len(n_item)):
    avg = {
        "source": collate_dic["net_input"]["src_tokens"][i, :src_lengths[i]].mean(-1).mean(), 
        "src_audio_path": collate_dic["net_input"]["src_audio_path"][i], 
        "target": collate_dic["target"][i, :target_lengths[i]].float().mean(), 
        "img_tensor": collate_dic["net_input"]["img_tensor"][i].mean(-1).mean(), 
        "img_list": collate_dic["net_input"]["imgs_list"][0][i].mean(-1).mean(), 
        "img_path": collate_dic["net_input"]["img_path"][i], 
        "source_unit": collate_dic["multitask"]["source_unit"]["target"][i, :collate_dic["multitask"]["source_unit"]["target_lengths"][i]].float().mean(), 
    }
    collate_inputs.append(avg)

print(order)
for i in range(len(collate_inputs)):
    print(order[i], collate_inputs[i])
print()

# print(collate_dic)

5 {'source': tensor(-3.6722e-08), 'src_audio_path': '/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/fr/test.2017/839.wav', 'target': tensor(486.4021), 'img_tensor': tensor(0.1274), 'img_list': tensor(0.0119), 'img_path': '/opt/data/private/dsy/project/dataset/flickr30k/test2017-images/21513580136_ea14fd4b8f.jpg', 'source_unit': tensor(456.5901)}
6 {'source': tensor(-1.3912e-07), 'src_audio_path': '/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/fr/test.2017/1.wav', 'target': tensor(501.3901), 'img_tensor': tensor(-0.0760), 'img_list': tensor(0.0165), 'img_path': '/opt/data/private/dsy/project/dataset/flickr30k/test2017-images/22439193117_4183a49ec2.jpg', 'source_unit': tensor(451.5310)}
7 {'source': tensor(-1.6040e-08), 'src_audio_path': '/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/fr/test.2017/321.wav', 'target': tensor(449.8269), 'img_tensor': tensor(-0.1060), 'img_list': tensor(0.0358), 'img_

: 

In [4]:
print(collate_dic.keys())

dict_keys(['id', 'net_input', 'speaker', 'target', 'target_lengths', 'ntokens', 'nsentences', 'multitask'])


# xlsr

In [1]:
import torch, torchaudio
import json
import IPython.display as ipd
import fairseq
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ckpt_path = "/opt/data/private/dsy/project/checkpoint/wav2vec2_fair/XLSR-53/xlsr_53_56k.pt"
# ckpt_path = "/opt/data/private/dsy/project/checkpoint/wav2vec2-FR-7K-base/checkpoint_best.pt"
ckpt_path = "/opt/data/private/dsy/project/checkpoint/mHuBERT/mhubert_base_vp_en_es_fr_it3.pt"
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
model = model[0]
model.to(device)
model.eval()
print()

/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-03 23:16:45 | INFO | fairseq.tasks.hubert_pretraining | current directory is /opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut/scripts
2023-07-03 23:16:45 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/checkpoint/annl/s2st/data/voxpopuli/mHuBERT/en_es_fr', 'fine_tuning': False, 'labels': ['km'], 'label_dir': '/checkpoint/wnhsu/experiments/hubert/kmeans/mhubert_vp_en_es_fr_it2_400k/en_es_fr.layer9.km500', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': 

In [2]:
print(cfg)
print()

{'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 200, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': 'tblog', 'wandb_project': None, 'azureml_logging': False, 'seed': 1337, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 

# wav2vec2 ctc

In [1]:
import speechbrain

/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch, torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from speech_to_speech_translation.text_cleaner.cleaners import (
    english_cleaners,
    transliteration_cleaners,
)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

# model_dir = "/opt/data/private/dsy/project/checkpoint/wav2vec2-base-10k-voxpopuli-ft-es"
# model_dir = "/opt/data/private/dsy/project/checkpoint/wav2vec2-large-xlsr-53-spanish"
# model_dir = "/opt/data/private/dsy/project/checkpoint/wav2vec2-large-xlsr-53-french"
# model_dir = "/opt/data/private/dsy/project/checkpoint/wav2vec2-base-10k-voxpopuli-ft-en"
model_dir = "/opt/data/private/dsy/project/checkpoint/wav2vec2-large-960h-lv60-self"
processor = Wav2Vec2Processor.from_pretrained(model_dir)
model = Wav2Vec2ForCTC.from_pretrained(model_dir)
model.to(device)

# audio_path = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/es/valid/1.wav"
# audio_path = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/fr/valid/1.wav"
audio_path = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/en/valid/1.wav"
audio = torchaudio.load(audio_path)[0]#.squeeze(0)

# input_values = processor(audio, return_tensors="pt").input_values
input_values = audio
input_values = input_values.to(device)
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.decode(predicted_ids[0])
print(transcription)
print(english_cleaners(transcription))
print(transliteration_cleaners(transcription))

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /opt/data/private/dsy/project/checkpoint/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


A GROUP OF MEN ARE LOADING COTTON ONTO A TRUCK
a group of men are loading cotton onto a truck
a group of men are loading cotton onto a truck


In [ ]:
print(model.wav2vec2.config)
print()

# wav2vec2 in checkpoint

In [5]:
import torch, torchaudio
import json, argparse
import IPython.display as ipd
import fairseq
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
user_dir = "/opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut"
fairseq.utils.import_user_module(argparse.Namespace(user_dir=user_dir))

ckpt_path = "/opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut/checkpoints/textless_es-en_wav2vec2/checkpoints/checkpoint_best.pt"
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
model = model[0]
model.to(device)
model.eval()
print()

2023-07-06 15:40:46 | INFO | fairseq.tasks.speech_to_speech | dictionary size: 1,004
2023-07-06 15:40:47 | INFO | mm_s2ut.models.mm_s2s_transformer | load_visual_extractor_type = None
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /opt/data/private/dsy/project/checkpoint/wav2vec2-base-es-voxpopuli-v2 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-07-06 15:40:48 | INFO | mm_s2ut.models.mm_s2s_transformer | loaded /opt/data/private/dsy/project/checkpoint/wav2vec2-base-es-voxpopuli-v2
2023-07-06 15:40:48 | INFO | mm_s2ut.models.mm_s2s_transformer | multimodal_translation_flag = False
2023-07-06 15:40:48 | INFO | mm_s2ut.models.mm_s2s_transformer | is_fusion_top = False


In [6]:
model.encoder.wav2vec2.training
sum_param = 0
for param in model.encoder.wav2vec2.parameters():
    sum_param += torch.sum(torch.abs(param))
    # print(param)
    # print(param.requires_grad)
print(sum_param)

tensor(5486716.5000, device='cuda:0', grad_fn=<AddBackward0>)


: 

In [1]:
import torch, torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

model_dir = "/opt/data/private/dsy/project/checkpoint/wav2vec2-base-es-voxpopuli-v2"
processor = Wav2Vec2Processor.from_pretrained(model_dir)
model = Wav2Vec2ForCTC.from_pretrained(model_dir)
model.to(device)
print()

/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /opt/data/private/dsy/project/checkpoint/wav2vec2-base-es-voxpopuli-v2 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
model.training
sum_param = 0
for param in model.parameters():
    sum_param += torch.sum(torch.abs(param))
print(sum_param)

tensor(5462147.5000, grad_fn=<AddBackward0>)


# noise dataset

In [1]:
import torch, torchaudio
import json, argparse
import IPython.display as ipd
import fairseq
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
user_dir = "/opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut"
fairseq.utils.import_user_module(argparse.Namespace(user_dir=user_dir))

ckpt_path = "/opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut/checkpoints/textless_fr-en_wav2vec2/checkpoints/checkpoint_best.pt"
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
model = model[0]
model.to(device)
model.eval()
print()

/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-12 01:59:14 | INFO | fairseq.tasks.speech_to_speech | dictionary size: 1,004
2023-07-12 01:59:15 | INFO | mm_s2ut.models.mm_s2s_transformer | load_visual_extractor_type = None
/opt/data/private/dsy/project/model/spoken_question_answering/transformers/src/transformers/models/wav2vec2/modeling_wav2vec2.py:1915: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(
2023-07-12 01:59:17 | INFO | mm_s2ut.models.mm_s2s_transformer | loaded /opt/data/private/dsy/project/checkpoint/wav2vec2-base-10k-voxpopuli-ft-es
2023-07-12 01:59:17 | INFO | mm_s2ut.models.mm_s2s_transfor

In [2]:
gen_subset = "train"
task.load_dataset(gen_subset)
dataset = task.dataset(gen_subset)

2023-07-12 01:59:26 | WARNING | fairseq.data.audio.data_cfg | Auto converting transforms into feature_transforms, but transforms will be deprecated in the future. Please update this in the config.
2023-07-12 01:59:26 | WARNING | fairseq.data.audio.speech_to_text_dataset | Feature transforms will not be applied. To use feature transforms, set use_audio_input as False in config.
2023-07-12 01:59:27 | INFO | fairseq.data.audio.speech_to_text_dataset | 'train' has 0.00% OOV
2023-07-12 01:59:27 | INFO | fairseq.data.audio.speech_to_text_dataset | MultiModalSpeechToSpeechDataset(split="train", n_samples=29_000, prepend_tgt_lang_tag=False, n_frames_per_step=1, shuffle=False, feature_transforms=CompositeAudioFeatureTransform(
    UtteranceCMVN(norm_means=True, norm_vars=True)
    SpecAugmentTransform(time_warp_w=0, freq_mask_n=1, freq_mask_f=27, time_mask_n=1, time_mask_t=100, time_mask_p=1.0)
), waveform_transforms=None, dataset_transforms=CompositeAudioDatasetTransform(
))
2023-07-12 01:59:2

In [3]:
import numpy as np
import wavfile
import soundfile as sf
task.noise_config
rand_indexes = np.random.randint(0, len(task.noise_config.noise_wav), size=task.noise_config.noise_num)
# rand_indexes = rand_indexes.astype(int)
for x in rand_indexes:
    x = int(x)
    print(task.noise_config.noise_wav[x])
    item = sf.read(task.noise_config.noise_wav[x])
    print(item[0][:,None].shape)

/opt/data/private/dsy/project/dataset/musan/music/fma-western-art/music-fma-wa-0037.wav
(2906070, 1)


In [27]:
item = dataset[0]
# if len(item) >= 2:
#     item = item[0]
item.src_audio_path
audio = item.source.cpu().numpy()
ipd.Audio(audio, rate=16_000)

In [6]:
audio

array([], dtype=float32)

In [14]:
wav = torchaudio.load(item.src_audio_path)[0]
ipd.Audio(wav, rate=16_000)

# wav2vec2 from huggingface and its processor

In [1]:
import sys
sys.path.append("/opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut/models")
import os, torch
from fuse import Wav2Vec2WithMultiModal
wav2vec2_dir = "/opt/data/private/dsy/project/checkpoint/wav2vec2-base-10k-voxpopuli-ft-fr"
wav2vec2_mm = Wav2Vec2WithMultiModal(
    wav2vec2_dir,
    num_cross_attention_layers=2,
    wav2vec2_embed_dim=768,
    m2_dim=768,
    dropout=0.1,
    batch_first=True,
)

/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/data/private/dsy/project/model/spoken_question_answering/transformers/src/transformers/models/wav2vec2/modeling_wav2vec2.py:1915: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [6]:
wav2vec2_mm.eval()
# wav2vec2_mm.train()
batch_size = 3
audio_length = 16_000 * 4
# input_values = torch.rand((batch_size, audio_length))
input_values = torch.full((batch_size, audio_length), fill_value=0.1)
attention_mask = torch.full_like(input_values, fill_value=True)
m2 = torch.full((batch_size, 50, 768), fill_value=0.3)
m2 = [m2] * 2
m2 = None
mm_outputs = wav2vec2_mm(
    input_values=input_values, attention_mask=attention_mask,
    m2=m2, m2_mask=None,
    output_hidden_states=True, return_dict=True,
)
print(mm_outputs["hidden_states"][-1].sum())

tensor(-393.2354, grad_fn=<SumBackward0>)


In [3]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained(wav2vec2_dir)
model_outputs = model(
    input_values=input_values, attention_mask=attention_mask,
    output_hidden_states=True, return_dict=True,
)
print(model_outputs["hidden_states"][-1].sum())

tensor(-393.2354, grad_fn=<SumBackward0>)


In [3]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torchaudio
import torch

# load model & processor
model = Wav2Vec2ForCTC.from_pretrained("/opt/data/private/dsy/project/checkpoint/wav2vec2-base-10k-voxpopuli-ft-fr")
processor = Wav2Vec2Processor.from_pretrained("/opt/data/private/dsy/project/checkpoint/wav2vec2-base-10k-voxpopuli-ft-fr")

In [ ]:
model
# model.wav2vec2.encoder

In [4]:
wav_root = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/fr/valid"
wav_path_list = [f"{wav_root}/{i}.wav" for i in range(1, 6)]
audio_list = [torchaudio.load(wav_path)[0][0].cpu().numpy() for wav_path in wav_path_list]
[print(audio.shape) for audio in audio_list]

inputs = processor(audio_list, sampling_rate=16_000, return_tensors="pt", padding=True, return_attention_mask=True)
outputs = model(
    **inputs, 
    output_attentions=True, 
    output_hidden_states=True, 
    return_dict=True, 
)
logits = outputs.logits
predicted_ids = torch.argmax(logits, axis=-1)
print(len(outputs["hidden_states"]), outputs["hidden_states"][-1].shape)
print(processor.batch_decode(predicted_ids))
print(type(outputs))

(51840,)
(57600,)
(63360,)
(69120,)
(76416,)
13 torch.Size([5, 238, 768])
['un groupe de homme charge du coton dans un camion', 'un homme dormant dans une chambre verte sur un canapa', 'un guerrçon avec un case qui est assi sur les épols de une femmes', 'deux hommes installant une tente de pêche sur glasble sur un lagu jele', 'un omme chauff vétud et un gilet de sauvetage rouge est aissi dans un petit bateaux']
<class 'transformers.modeling_outputs.CausalLMOutput'>


In [6]:
print(outputs["hidden_states"][-1].shape)
print(outputs.keys())
print(inputs["attention_mask"].shape)

torch.Size([5, 238, 768])
odict_keys(['logits', 'hidden_states', 'attentions'])
torch.Size([5, 76416])


In [18]:
~(inputs['attention_mask'].bool())
# inputs['input_values'].shape
# inputs['attention_mask'].shape

tensor([[False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ..., False, False, False]])

In [5]:
print(inputs.keys())
inputs["input_values"][:, -10:]
# print(inputs["input_values"][0][51840-20:51840])
print(inputs["attention_mask"].sum(-1))
inputs["attention_mask"]
# outputs.keys()

dict_keys(['input_values', 'attention_mask'])
tensor([51840, 57600, 63360, 69120, 76416])


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)

In [9]:
def lengths_to_padding_mask(lens):
    bsz, max_lens = lens.size(0), torch.max(lens).item()
    mask = torch.arange(max_lens).to(lens.device).view(1, max_lens)
    mask = mask.expand(bsz, -1) >= lens.view(bsz, 1).expand(-1, max_lens)
    return mask

padding_mask = lengths_to_padding_mask(torch.tensor([audio.shape[0] for audio in audio_list]))
# attention_mask = torch.tensor(~padding_mask, dtype=torch.int32, device=padding_mask.device)
attention_mask = (~padding_mask).int()
# attention_mask = ~padding_mask

In [10]:
attention_mask
attention_mask.sum(-1)

tensor([51840, 57600, 63360, 69120, 76416])

In [13]:
outputs = model(
    input_values=inputs["input_values"],
    attention_mask=attention_mask, 
    output_attentions=True, 
    output_hidden_states=True, 
    return_dict=True, 
)
logits = outputs.logits
predicted_ids = torch.argmax(logits, axis=-1)
print(processor.batch_decode(predicted_ids))

['un groupe de homme charge du coton dans un camion', 'un homme dormant dans une chambre verte sur un canapa', 'un guerrçon avec un case qui est assi sur les épols de une femmes', 'deux hommes installant une tente de pêche sur glasble sur un lagu jele', 'un omme chauff vétud et un gilet de sauvetage rouge est aissi dans un petit bateaux']


In [14]:
outputs.keys()

odict_keys(['logits', 'hidden_states', 'attentions'])

In [15]:
def forward_padding_mask(
    features: torch.Tensor,
    padding_mask: torch.Tensor,
) -> torch.Tensor:
    extra = padding_mask.size(1) % features.size(1)
    if extra > 0:
        padding_mask = padding_mask[:, :-extra]
    padding_mask = padding_mask.view(padding_mask.size(0), features.size(1), -1)
    padding_mask = padding_mask.all(-1)
    return padding_mask

encoder_padding_mask = forward_padding_mask(
    outputs["hidden_states"][-1], 
    padding_mask, 
)

In [21]:
encoder_padding_mask
encoder_padding_mask.shape
outputs["attentions"][-1].shape

torch.Size([5, 12, 238, 238])

# speechmatrix noise

In [1]:
import sys
sys.path.append("/opt/data/private/dsy/project/model/multimodal_S2UT")
import torch, torchaudio
import librosa
import IPython.display as ipd
from mm_s2ut.data.audio_utils import add_noise_v2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wav_root = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/en/valid"
wav_path = f"{wav_root}/1.wav"
noise_root = "/opt/data/private/dsy/project/dataset/musan/music/fma-western-art"
noise_path = f"{noise_root}/music-fma-wa-0009.wav"

wav, wav_sr = torchaudio.load(wav_path)
noise, noise_sr = torchaudio.load(noise_path)
# noise, noise_sr = librosa.load(noise_path)
# wav, noise = noise, wav
print(wav.shape, wav_sr)
print(noise.shape, noise_sr)

torch.Size([1, 57600]) 16000
torch.Size([1, 3818057]) 16000


In [7]:
noisy_wav = add_noise_v2(
    waveforms=wav, 
    # noise_waveform=noise[:, 48_000:], 
    noise_waveform=noise[:, :], 
    snr_low=0, 
    snr_high=20, 
    noise_waveform_start=-1, 
    add_white_noise=False, 
    normalize=True, 
)
ipd.Audio(noisy_wav, rate=wav_sr)

# MultimodalTransformerEncoder

In [ ]:
import sys
sys.path.append("/opt/data/private/dsy/project/model/multimodal_S2UT")
import torch, torchaudio, torchsummary
import librosa
import IPython.display as ipd
from mm_s2ut.data.audio_utils import add_noise_v2
from mm_s2ut.models.fuse import MultimodalTransformerEncoder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
a = [0,1,2,3]
a[-3:0]

[]

# google vit

In [1]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

model_dir = "/opt/data/private/dsy/project/checkpoint/vit-base-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_dir)
model = ViTForImageClassification.from_pretrained(model_dir)

/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image_paths = [
    "/opt/data/private/dsy/project/checkpoint/vit-base-patch16-224/pic/000000039769.jpg",
    "/opt/data/private/dsy/project/dataset/flickr30k/test2017-images/8085680939_6c56b595ea.jpg"
]
images = [Image.open(image_path) for image_path in image_paths]

inputs = processor(images=images, return_tensors="pt")
# outputs = model(**inputs, output_hidden_states=True, return_dict=True)
outputs = model(pixel_values=inputs["pixel_values"], output_hidden_states=True, return_dict=True)
logits = outputs.logits
for i in range(logits.shape[0]):
    predicted_class_idx = logits[i].argmax(-1).item()
    print("Predicted class:", model.config.id2label[predicted_class_idx])

Predicted class: Egyptian cat
Predicted class: lakeside, lakeshore


In [3]:
print(inputs.keys())
print(outputs.keys())
print(inputs['pixel_values'].shape)
print(outputs["logits"].shape)
print(len(outputs['hidden_states']), outputs['hidden_states'][-3].shape)
# print(image.size, inputs['pixel_values'].shape)

dict_keys(['pixel_values'])
odict_keys(['logits', 'hidden_states'])
torch.Size([2, 3, 224, 224])
torch.Size([2, 1000])
13 torch.Size([2, 197, 768])


In [4]:
import torch
pth_path = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/image_feat/vit_base_patch16_384/valid.pth"
pth = torch.load(pth_path)
print(pth.shape)

torch.Size([1014, 577, 768])


In [17]:
a = [
    [0, 100],
    [1, 101],
    [2, 102]
]
order = [2, 1, 0]
t_a = torch.tensor(a)
t_order = torch.tensor(order)
# t_order = t_order.unsqueeze(0)
print(t_a)
print(t_a.index_select(0, t_order))
print([a[i] for i in order])

tensor([[  0, 100],
        [  1, 101],
        [  2, 102]])
tensor([[  2, 102],
        [  1, 101],
        [  0, 100]])
[[2, 102], [1, 101], [0, 100]]


timm

In [1]:
import os
import torch
import timm
from PIL import Image

# model_dir = "/opt/data/private/dsy/project/checkpoint/timm/vit_base_patch16_clip_224.laion2b_ft_in12k"
model_dir = "/opt/data/private/dsy/project/checkpoint/timm/vit_base_patch16_clip_384.laion2b_ft_in12k_in1k"
model_name = os.path.split(model_dir)[-1]
model_path = os.path.join(model_dir, "pytorch_model.bin")
image_paths = [
    "/opt/data/private/dsy/project/checkpoint/vit-base-patch16-224/pic/000000039769.jpg",
    "/opt/data/private/dsy/project/dataset/flickr30k/test2017-images/8085680939_6c56b595ea.jpg"
]

model = timm.create_model(
    model_name,
    pretrained=False,
)
timm.models.load_checkpoint(model, model_path)
model = model.eval()
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_config

{'input_size': (3, 384, 384),
 'interpolation': 'bicubic',
 'mean': (0.48145466, 0.4578275, 0.40821073),
 'std': (0.26862954, 0.26130258, 0.27577711),
 'crop_pct': 1.0,
 'crop_mode': 'squash'}

In [3]:
img_tensor = []
for image_path in image_paths:
    img = Image.open(image_path)
    img_tensor.append(transforms(img))
img_tensor = torch.stack(img_tensor, dim=0)
# output = model(img_tensor)
output = model.forward_features(img_tensor)

In [ ]:
print(output.shape)
output
# img_tensor.shape

# open_ai clip

In [1]:
import os
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

# model_dir = "/opt/data/private/dsy/project/checkpoint/CLIP-ViT-B-32-laion2B-s34B-b79K"
# model_dir = "/opt/data/private/dsy/project/checkpoint/clip-vit-base-patch32"
model_dir = "/opt/data/private/dsy/project/checkpoint/clip-vit-base-patch16"
model_name = os.path.split(model_dir)[-1]
model_path = os.path.join(model_dir, "pytorch_model.bin")
model = CLIPModel.from_pretrained(model_dir)
processor = CLIPProcessor.from_pretrained(model_dir)

image_paths = [
    "/opt/data/private/dsy/project/checkpoint/vit-base-patch16-224/pic/000000039769.jpg",
    "/opt/data/private/dsy/project/dataset/flickr30k/test2017-images/8085680939_6c56b595ea.jpg"
]
images = [Image.open(image_path) for image_path in image_paths]

inputs = processor(images=images, return_tensors="pt")

outputs = model.vision_model(
    pixel_values=inputs["pixel_values"],
    output_hidden_states=True,
    return_dict=True,
)

/opt/data/private/dsy/project/opt/anaconda3/envs/mt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(inputs["pixel_values"].shape)
print(outputs.keys())
print(outputs["last_hidden_state"].shape, len(outputs["hidden_states"]), outputs["hidden_states"][-2].shape)
print(outputs["pooler_output"].shape)

torch.Size([2, 3, 224, 224])
odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])
torch.Size([2, 197, 768]) 13 torch.Size([2, 197, 768])
torch.Size([2, 768])


In [17]:
# a = outputs["last_hidden_state"]
b = outputs["last_hidden_state"]

In [18]:
print(a.sum())
print(b.sum())

tensor(-4777.8745, grad_fn=<SumBackward0>)
tensor(901.4413, grad_fn=<SumBackward0>)


# mhubert

In [7]:
import torch, torchaudio
from speech_to_speech_translation.mhubert import HubertCode
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hubert_dir = "/opt/data/private/dsy/project/checkpoint/mhubert-base"
hubert = HubertCode(
    hubert_model=hubert_dir,
    km_path=f"{hubert_dir}/mhubert_base_vp_en_es_fr_it3_L11_km1000.bin",    
)
hubert.to(device)
print()

In [8]:
wav_path = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/fr/valid/1.wav"
wav, sr = torchaudio.load(wav_path)
wav = wav.to(device)
feature = hubert(wav)["feature"]
code_dict = hubert.decode(feature)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 wav, sr = torchaudio.load(wav_path)                                                          │
│   3 wav = wav.to(device)                                                                         │
│   4 feature = hubert(wav)["feature"]                                                             │
│ ❱ 5 code_dict = hubert.decode(feature)                                                           │
│   6                                                                                              │
│                                                                                                  │
│ /opt/data/private/dsy/project/model/multimodal_S2UT/mm_s2ut/scripts/speech_to_speech_translation │
│ /mhubert.py:50 in decode                                                                         │
│                                                                                                  │
│   47 │                                                                                           │
│   48 │   def decode(self, feature, beamsearch=True, top_k=10, beamsize=200):                     │
│   49 │   │   dist = torch.sqrt(                                                                  │
│ ❱ 50 │   │   │   feature.pow(2).sum(1, keepdim=True)                                             │
│   51 │   │   │   - 2 * torch.matmul(feature, self.C)                                             │
│   52 │   │   │   + self.Cnorm                                                                    │
│   53 │   │   )                                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: The size of tensor a (768) must match the size of tensor b (1000) at non-singleton dimension 2

# 计算bleu提升

* fr-en

In [17]:
import torch
ori_bleu = [33.08, 35.14, 28.71, 25.32]
# res_bleu = [33.97, 33.53, 29.74, 26.11] # 0.27500009536743164
res_bleu = [33.86, 36.02, 30.24, 25.49] # 0.8400001525878906 47
# res_bleu = [34.34, 35.49, 30.05, 25.03] # 0.6650004386901855
# res_bleu = [33.67, 35.81, 30.60, 25.34] # 0.7925000190734863
enh_bleu = (torch.tensor(res_bleu) - torch.tensor(ori_bleu)).sum().item()
print(enh_bleu / 4)

0.8400001525878906


* es-en

In [9]:
import torch
ori_bleu = [40.70, 41.11, 37.07, 33.66]
res_bleu = []
res_bleu = [40.97, 41.52, 37.00, 34.95]
enh_bleu = (torch.tensor(res_bleu) - torch.tensor(ori_bleu)).sum().item()
print(enh_bleu / 4)

0.47500038146972656


* en-fr

In [ ]:
import torch
ori_bleu = [25.52, 26.74, 23.46, 20.07]
res_bleu = []
enh_bleu = (torch.tensor(res_bleu) - torch.tensor(ori_bleu)).sum().item()
print(enh_bleu / 4)

* en-es

In [12]:
import torch
# ori_bleu = [56.59, 56.65, 50.59, 47.69]
ori_bleu = [55.38, 56.58, 49.08, 46.69]
res_bleu = [56.89, 57.24, 50.91, 47.86]
enh_bleu = (torch.tensor(res_bleu) - torch.tensor(ori_bleu)).sum().item()
print(enh_bleu / 4)

1.2924995422363281


In [13]:
def get_perceiver(l_u, l_v, d):
    s = 20 * l_u * d * d + 4 * l_v * d * d + 4 * l_u * l_v * d + 3 * l_u * d
    return s

def get_ca(l_u, l_v, d):
    s = 2 * l_u * d * d + 4 * l_v * d * d +4 * l_u * l_v * d + 2 * l_u * d * d + l_u * d
    return s

# perceiver
p = 3 * get_perceiver(32, 200, 768) + 3 * get_perceiver(32, 197, 768)
p = get_perceiver(32, 197, 768)
# p = get_ca(32, 197, 768)
# ca
ca = get_ca(200, 197, 768)
print(p)
print(ca)

861708288
1057830912
